**Set environment**

In [ ]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

In [2]:
library("DESeq2")

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:gridExtra’:

    combine


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:dplyr’:

    first, rename


The following object is masked from ‘

## Import data

In [3]:
PREFIX = "KS91_K562_ASTARRseq"
FOLDER = "coverage_astarrseq_peak_macs_input"

In [4]:
fdiry = file.path(FD_RES, PREFIX, FOLDER, "summary")

fname = "wgs_count_matrix.tsv"
fpath = file.path(fdiry, fname)
dat_count = read_tsv(fpath)

fname = "wgs_metadata.tsv"
fpath = file.path(fdiry, fname)
dat_meta = read_tsv(fpath)

Rows: 246852 Columns: 13
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (1): Chrom
dbl (12): Start, End, Input_rep1, Input_rep2, Input_rep3, Input_rep4, Input_...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 10 Columns: 3
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): Sample, Group, FPath

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


**Arrange count matrix and metadata**

In [5]:
dat_col = dat_meta  %>% 
    dplyr::select(Sample, Group) %>% 
    dplyr::rename(condition = Group) %>%
    column_to_rownames(var = "Sample")

dat_cnt = dat_count %>% 
    dplyr::mutate(Peak = paste(Chrom, Start, End, sep = "_")) %>%
    dplyr::select(-Chrom, -Start, -End) %>%
    column_to_rownames(var = "Peak")

dat_cnt[is.na(dat_cnt)] = 0

In [6]:
head(dat_cnt)

,Input_rep1,Input_rep2,Input_rep3,Input_rep4,Input_rep5,Input_rep6,Output_rep1,Output_rep2,Output_rep3,Output_rep4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1_10015_10442,155,214,257,236,185,214,7,12,4,11
chr1_14253_14645,110,144,160,141,130,130,8,26,30,57
chr1_16015_16477,141,208,206,190,202,182,9,9,18,23
chr1_17237_17772,259,350,399,367,369,331,7,13,23,59
chr1_28903_29613,263,338,368,333,352,317,12,18,3,32
chr1_30803_31072,82,115,171,136,105,115,13,22,14,33


In [7]:
dat_col

,condition
,<chr>
Input_rep1,Input
Input_rep2,Input
Input_rep3,Input
Input_rep4,Input
Input_rep5,Input
Input_rep6,Input
Output_rep1,Output
Output_rep2,Output
Output_rep3,Output


In [8]:
print(all(rownames(dat_col) %in% colnames(dat_cnt)))
print(all(rownames(dat_col) ==   colnames(dat_cnt)))

[1] TRUE
[1] TRUE


## Run DESeq2

In [9]:
dds = DESeqDataSetFromMatrix(
    countData = dat_cnt, 
    colData   = dat_col, 
    design    = ~condition)

converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


In [10]:
### remove the peaks which have < 10 reads
dds = dds[rowSums(counts(dds)) >= 10,]

### set control condition as reference
dds$condition <- relevel(dds$condition, ref = "Input")

In [11]:
dds = DESeq(dds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



## Get results

In [12]:
resultsNames(dds)

[1] "Intercept"                 "condition_Output_vs_Input"

In [13]:
res = results(dds)
res

log2 fold change (MLE): condition Output vs Input 
Wald test p-value: condition Output vs Input 
DataFrame with 246850 rows and 6 columns
                          baseMean log2FoldChange     lfcSE      stat
                         <numeric>      <numeric> <numeric> <numeric>
chr1_10015_10442           71.4038      -2.213613  0.269152  -8.22441
chr1_14253_14645           68.6303       0.222186  0.153410   1.44831
chr1_16015_16477           71.2541      -1.271295  0.201963  -6.29470
chr1_17237_17772          124.0313      -1.385289  0.163863  -8.45396
chr1_28903_29613          113.6310      -1.942200  0.188613 -10.29727
...                            ...            ...       ...       ...
chrX_156000382_156003205 1532.3823       0.468703 0.0375633  12.47767
chrX_156009687_156010227   59.3053      -1.562456 0.2365061  -6.60641
chrX_156016391_156016836   57.0261      -3.944716 0.4857124  -8.12151
chrX_156024950_156025593  164.6780      -0.401819 0.1139605  -3.52595
chrX_156030187_1560307

In [14]:
res = results(dds)
res = as.data.frame(res) %>% rownames_to_column(var = "Peak")
head(res)

,Peak,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,chr1_10015_10442,71.40375,-2.2136128,0.2691517,-8.2244060,1.961605e-16,2.910689e-15
2,chr1_14253_14645,68.63035,0.2221859,0.1534101,1.4483134,1.475294e-01,2.178106e-01
3,chr1_16015_16477,71.25409,-1.2712950,0.2019628,-6.2946978,3.079993e-10,2.653010e-09
4,chr1_17237_17772,124.03128,-1.3852891,0.1638628,-8.4539594,2.815873e-17,4.431894e-16
5,chr1_28903_29613,113.63103,-1.9422003,0.1886131,-10.2972733,7.248682e-25,1.723665e-23
6,chr1_30803_31072,57.37875,-0.1332069,0.2145692,-0.6208109,5.347240e-01,6.231400e-01


## Save results

In [15]:
fdiry = file.path(FD_RES, PREFIX, FOLDER, "summary")
fname = "wgs_deseq_results.tsv"
fpath = file.path(fdiry, fname)
print(fpath)

write_tsv(res, fpath)

[1] "/mount/work/out/proj_combeffect_encode_fcc/KS91_K562_ASTARRseq/coverage_astarrseq_peak_macs_input/summary/wgs_deseq_results.tsv"
